# T1007 - System Service Discovery
Adversaries may try to gather information about registered local system services. Adversaries may obtain information about services using tools as well as OS utility commands such as <code>sc query</code>, <code>tasklist /svc</code>, <code>systemctl --type=service</code>, and <code>net start</code>.

Adversaries may use the information from [System Service Discovery](https://attack.mitre.org/techniques/T1007) during automated discovery to shape follow-on behaviors, including whether or not the adversary fully infects the target and/or attempts specific actions.

## Atomic Tests

### Atomic Test #1 - System Service DiscoveryIdentify system services.

Upon successful execution, cmd.exe will execute service commands with expected result to stdout.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
tasklist.exe
sc query
sc query state= all
```

In [ ]:
Invoke-AtomicTest T1007 -TestNumbers 1

### Atomic Test #2 - System Service Discovery - net.exeEnumerates started system services using net.exe and writes them to a file. This technique has been used by multiple threat actors.

Upon successful execution, net.exe will run from cmd.exe that queries services. Expected output is to a txt file in in the temp directory called service-list.txt.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
net.exe start >> %temp%\service-list.txt
```

In [ ]:
Invoke-AtomicTest T1007 -TestNumbers 2

#### Cleanup: 
```cmd
del /f /q /s %temp%\service-list.txt >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1007 -TestNumbers 2 -Cleanup

### Atomic Test #3 - System Service Discovery - systemctl/serviceEnumerates system service using systemctl/service
**Supported Platforms:** linux#### Attack Commands: Run with `bash`
```bash
if [ "$(uname)" = 'FreeBSD' ]; then service -e; else systemctl --type=service; fi;
```

In [ ]:
Invoke-AtomicTest T1007 -TestNumbers 3

## Detection
System and network discovery techniques normally occur throughout an operation as an adversary learns the environment. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities, such as Lateral Movement, based on the information obtained.

Monitor processes and command-line arguments for actions that could be taken to gather system information related to services. Remote access tools with built-in features may interact directly with the Windows API to gather information. Information may also be acquired through Windows system management tools such as [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047) and [PowerShell](https://attack.mitre.org/techniques/T1059/001).


## Shield Active Defense
### API Monitoring 
 Monitor local APIs that might be used by adversary tools and activity. 

 API Monitoring involves capturing an internal Operating System (OS) function for its usage, accompanying arguments, and result. When a defender captures this information, the intelligence gathered can be analyzed to gain insight into the activity of an adversary at a level deeper than normal system activity monitoring.
#### Opportunity
There is an opportunity for the defender to observe the adversary and control what they can see, what effects they can have, and/or what data they can access.
#### Use Case
A defender can monitor and analyze operating system functions calls for detection and alerting.
#### Procedures
Trace activity through WinSock TCP API functions to view potentially  malicious network events. Log it such that it can be pushed to a centralized location and analyzed further.
Hook the Win32 DeleteFile() function to log all attempts at deleting a given file. This information can be used to trigger restoration attempts on critical data, reducing potential disruption if those files are unavailable for prolonged periods of time.
